In [1]:
from joblib import load
import os
import json
import numpy as np
from pathlib import Path
import os
import pandas as pd

In [2]:
curation_root = Path("/mnt/i/bio/mave_curation/")
dataset_paths = list(map(lambda pth: pth.parent, curation_root.glob("*/observations.pkl")))

In [3]:
import logging
def read_dataset(path,return_counts=False):
    observations = load(path / "observations.pkl")
    hgvs_pro = load(path / "hgvs_pro.pkl")
    for k, v in hgvs_pro.items():
        if len(v) > len(set(v)):
            logging.warning(f"Duplicate HGVS_pro in {path} - {k} - n duplicates {len(v) - len(set(v))}")
    if return_counts:
        return {k : len(v) for k,v in observations.items()}
    return observations, hgvs_pro

In [4]:
stats = {str(pth.stem.replace("_"," - ")) : read_dataset(pth,return_counts=True) for pth in dataset_paths}

In [5]:
pd.DataFrame(stats).T.to_excel("dataset_stats.xlsx")

In [7]:
datasets = {str(pth.stem.replace("_"," - ")) : read_dataset(pth,return_counts=False) for pth in dataset_paths}

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
nsamples = 6
fig,axs = plt.subplots(len(datasets),nsamples,figsize=(nsamples*7,7 * len(datasets)),sharex='row')
for ax,(ds_name, data) in zip(axs,datasets.items()):
    nsamples = len(data[0])
    for i, (k,v) in enumerate(data[0].items()):
        ax[i].hist(v,label=f"n={len(v)}")
        ax[i].legend()
        ax[i].set_title(f"Dataset: {ds_name}\nSample: {k}")

In [9]:
fig.savefig("dataset_histograms.pdf",bbox_inches='tight',format='pdf')